In [ ]:
import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt
from scipy.spatial import distance
plt.rc('figure', figsize=(8, 5))
import helpers as h

In [ ]:
importlib.reload(h)

In [ ]:
def edge_exists(cycle, i, j):
    for x in range(len(cycle) - 1):
        if (cycle[x], cycle[x+1]) == (i, j) or (cycle[x], cycle[x+1]) == (j , i):
            return True
        if (cycle[-1], cycle[0]) == (i, j) or (cycle[-1], cycle[0]) == (j , i):
            return True
    return None

def any_edge_exist(cycle1, cycle2, i, j):
    for id, c in enumerate([cycle1, cycle2]):
        exist = edge_exists(c, i, j)
        if exist:
            return id, exist
    return None, None

def new_moves(cycle1, cycle2, distances, move):
    _, type_of_transformation, id, _, i, _, _, j, _ = move
    moves = []
    if type_of_transformation == 'edge':
        c = [cycle1, cycle2][id]
        moves = []
        for v in [i,j]:
            z = np.where(np.array(c)  == v)[0][0]
            for x in range(len(c)):
                if x == z:
                    continue
                d = h.change_edges_inside(c, z, x, distances)
                if d < -0.001:
                    x1, y1, z1 = c[(z-1) % len(c)], c[z], c[(z + 1) % len(c)]
                    x2, y2, z2 = c[(x-1) % len(c)], c[x], c[(x+1) % len(c)]
                    moves.append((d, 'edge', id, x1, y1, z1, x2, y2, z2))
    else:
        i_index = np.where(np.array(cycle1)  == j)[0][0]
        j_index = np.where(np.array(cycle2)  == i)[0][0]
        for x in range(len(cycle1)):
            c1, c2, d = h.change_vertices(cycle1, cycle2, i_index, x, distances)
            if d < -0.001:
                x1, y1, z1 = cycle1[(i_index-1) % len(cycle1)], cycle1[i_index], cycle1[(i_index + 1) % len(cycle1)]
                x2, y2, z2 = cycle2[(x-1) % len(cycle2)], cycle2[x], cycle2[(x+1) % len(cycle2)]
                moves.append((d, 'vertex', None, x1, y1, z1, x2, y2, z2))
        for x in range(len(cycle2)):
            c1, c2, d = h.change_vertices(cycle2, cycle1, j_index, x, distances)
            if d < -0.001:
                x1, y1, z1 = cycle1[(x-1) % len(cycle1)], cycle1[x], cycle1[(x + 1) % len(cycle1)]
                x2, y2, z2 = cycle2[(j_index-1) % len(cycle2)], cycle2[j_index], cycle2[(j_index+1) % len(cycle2)]
                moves.append((d, 'vertex', None, x1, y1, z1, x2, y2, z2))
    return moves


def lm_search(points, distances):
    cycle1, cycle2 = h.random_solution(points, distances)
    lm = sorted(h.get_moves(cycle1, cycle2, distances), key=lambda move: move[0])
    while True:
        performed_move = None
        for move in list(lm):
            _, type_of_transformation, id, x1, y1, z1, x2, y2, z2 = move
            if type_of_transformation == 'vertex':
                e1 = edge_exists(cycle1, x1, y1)
                e2 = edge_exists(cycle1, y1, z1)
                e3 = edge_exists(cycle2, x2, y2)
                e4 = edge_exists(cycle2, y2, z2)
                if e1 is None or e2 is None or e3 is None or e4 is None:
                    lm.remove(move)
                elif e1 == e2 and e3==e4:
                    lm.remove(move)
                    performed_move = move
                    break
            if type_of_transformation == 'edge':
                c1, e1 = any_edge_exist(cycle1, cycle2, y1, z1)
                c2, e2 = any_edge_exist(cycle1, cycle2, y2, z2)
                if c1 != c2 or e1 is None or e2 is None:
                    lm.remove(move)
                elif e1 == e2 == True:
                    lm.remove(move)
                    performed_move = move
                    break
                elif e1 == e2 == False:
                    lm.remove(move)
                    _, type_of_transformation, id, x1, y1, z1, x2, y2, z2 = move
                    performed_move = move[0], 'edge', id, x1, z1, y1, x2, z2, y2
                    break

        if performed_move is None:
            break
        else:
            cycle1, cycle2  = h.apply_move(performed_move, cycle1, cycle2, distances)

        n_moves = new_moves(cycle1, cycle2, distances, performed_move)
        lm = sorted(list(set(lm).union(set(n_moves))), key=lambda x: x[0])
    return cycle1, cycle2


def candidate_search(points, distances):
    cycle1, cycle2 = h.random_solution(points, distances)
    closest = np.argpartition(distances, 8, axis=1)[:,:8]
    while True:
        cycles = [cycle1, cycle2]
        performed_move, best_delta = None, -0.001
        for v1 in range(len(distances)):
            for v2 in closest[v1]:
                if v1 == v2: continue
                c1, i = h.get_cycle_index(cycles, v1)
                c2, j = h.get_cycle_index(cycles, v2)
                move, delta = None, None
                if c1 == c2:
                    cycle = cycles[c1]
                    delta = h.change_edges_inside(cycle, i, j, distances)
                    x1, y1, z1 = cycle[(i-1) % len(cycle)], cycle[i], cycle[(i + 1) % len(cycle)]
                    x2, y2, z2 = cycle[(j-1) % len(cycle)], cycle[j], cycle[(j + 1) % len(cycle)]
                    move = delta, 'edge', c1, x1, y1, z1, x2, y2, z2
                else:
                    _, _, delta = h.change_vertices(cycles[c1], cycles[c2], i, j, distances)
                    x1, y1, z1 = cycles[c1][(i-1) % len(cycle1)], cycles[c1][i], cycles[c1][(i + 1) % len(cycle1)]
                    x2, y2, z2 = cycles[c2][(j-1) % len(cycle2)], cycles[c2][j], cycles[c2][(j + 1) % len(cycle2)]
                    move = delta, 'vertex', None, x1, y1, z1, x2, y2, z2
                if delta < best_delta:
                    best_delta = delta
                    performed_move = move

        if performed_move is None:
            break
        cycle1, cycle2 = h.apply_move(performed_move, cycle1, cycle2, distances)
    return cycles[0], cycles[1]


In [ ]:
importlib.reload(h)

In [ ]:
import time

results = {
    ('regret_method' , 'kroA200.tsp') : [],
    ('steepest_search', 'kroA200.tsp')  : [],
    ('lm_search', 'kroA200.tsp')  : [],
    ('candidate_search', 'kroA200.tsp') : [],

    ('regret_method', 'kroB200.tsp')  : [],
    ('steepest_search', 'kroB200.tsp')  : [],
    ('lm_search', 'kroB200.tsp')  : [],
    ('candidate_search', 'kroB200.tsp') : []
}

time_val = {
    ('regret_method', 'kroA200.tsp') : [],
    ('steepest_search', 'kroA200.tsp')  : [],
    ('lm_search', 'kroA200.tsp') : [],
    ('candidate_search', 'kroA200.tsp'): [],

    ('regret_method', 'kroB200.tsp') :  [],
    ('steepest_search', 'kroB200.tsp')  : [],
    ('lm_search', 'kroB200.tsp') : [],
    ('candidate_search', 'kroB200.tsp'): []
}

min_cycles = {
    ('regret_method', 'kroA200.tsp')  : [None, None],
    ('steepest_search', 'kroA200.tsp')  : [None, None],
    ('lm_search', 'kroA200.tsp')  : [None, None],
    ('candidate_search', 'kroA200.tsp') : [None, None],

    ('regret_method', 'kroB200.tsp')  : [None, None],
    ('steepest_search', 'kroB200.tsp')  : [None, None],
    ('lm_search', 'kroB200.tsp')  : [None, None],
    ('candidate_search', 'kroB200.tsp') : [None, None]
}
from helpers import steepest_search
from tqdm import tqdm
for i in tqdm(range(100)):
    for file in ['kroA200.tsp', 'kroB200.tsp']:
        points = h.load_file(file)
        distances = h.get_distances(points)

        start = time.time()
        cycle1, cycle2 = h.regret_method(points, distances)
        end = time.time()
        time_val[('regret_method', file)].append(end-start)
        c_len = h.cycle_length(distances, cycle1) + h.cycle_length(distances, cycle2)
        results[('regret_method', file)].append(c_len)

        if min_cycles[('regret_method', file)][0] is None or min_cycles[('regret_method', file)][0] > c_len:
            min_cycles[('regret_method', file)][0] = c_len
            min_cycles[('regret_method', file)][1] = [cycle1, cycle2]

        for met in [steepest_search, lm_search, candidate_search]:
            met_name = [name for name in globals() if globals()[name] is met][0]
            points_copy = points.copy()
            start = time.time()
            c1, c2 = met(points_copy, distances)
            end = time.time()
            cycles_len = h.cycle_length(distances, c1) + h.cycle_length(distances, c2)
            if min_cycles[(met_name, file)][0] is None or min_cycles[(met_name, file)][0] > cycles_len:
                min_cycles[(met_name, file)][0] = cycles_len
                min_cycles[(met_name, file)][1] = [c1, c2]
            results[(met_name, file)].append(cycles_len)
            time_val[(met_name, file)].append(end-start)
for key, value in min_cycles.items():
    print(key, value[0])
    h.draw_path(h.load_file(key[1]), value[1][0], value[1][1], key[1] + "_" + key[0] + ".jpg")
pd.DataFrame(results).describe()

In [ ]:
pd.DataFrame(time_val).describe()

In [ ]:
pd.DataFrame(results).describe()